The purpose of this notebook is to explore TensorFlow in 3 small projects to help understand the library and how to use it to get strong results with neural networks.

Due to this, not a lot of data cleaning or feature engineering will be done.

In [326]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import random

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.utils.np_utils import to_categorical

<h1>Titanic</h1>

First, I will use the Sequential model in Keras to compete in this competetion on Kaggle:

https://www.kaggle.com/c/titanic/overview

It's the quintessential beginner machine learning project, and is tabular data, so it seemed like a good place to start.

In [95]:
titanic_train = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/titanic/train.csv')
titanic_test = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/titanic/test.csv')

titanic_submission = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/titanic/gender_submission.csv')

In [96]:
titanic_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [97]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [98]:
titanic_test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [99]:
titanic_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [100]:
titanic_submission.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [101]:
MEAN_AGE = np.nanmean(list(titanic_train['Age']) + list(titanic_test['Age']))

titanic_train['Age'].fillna(MEAN_AGE, inplace=True)
titanic_test['Age'].fillna(MEAN_AGE, inplace=True)

MEAN_FARE = np.nanmean(list(titanic_train['Fare']) + list(titanic_test['Fare']))
titanic_train['Age'].fillna(MEAN_FARE, inplace=True)
titanic_test['Age'].fillna(MEAN_FARE, inplace=True)

In [102]:
titanic_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], 1, inplace=True)
titanic_test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], 1, inplace=True)

titanic_train['Pclass'] = titanic_train['Pclass'].astype('str')
titanic_test['Pclass'] = titanic_train['Pclass'].astype('str')

titanic_train = pd.get_dummies(titanic_train, drop_first=True)
titanic_test = pd.get_dummies(titanic_test, drop_first=True)

In [103]:
scaler = MinMaxScaler()
titanic_train = scaler.fit_transform(titanic_train).T
titanic_test = scaler.fit_transform(titanic_test)
# This is a lazy, non-professional approach since the train and test columns might be scaled slightly different.

X_train = titanic_train[1:]
y_train = titanic_train[0]
X_train = X_train.T

In [105]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20)
preds = model.predict(titanic_test)

Train on 891 samples
Epoch 1/20
891/891 [==============================] - 0s 203us/sample - loss: 0.6060 - acc: 0.6352
Epoch 2/20
891/891 [==============================] - 0s 66us/sample - loss: 0.5414 - acc: 0.7609
Epoch 3/20
891/891 [==============================] - 0s 67us/sample - loss: 0.5114 - acc: 0.7755
Epoch 4/20
891/891 [==============================] - 0s 71us/sample - loss: 0.4821 - acc: 0.7856
Epoch 5/20
891/891 [==============================] - 0s 70us/sample - loss: 0.4771 - acc: 0.7924
Epoch 6/20
891/891 [==============================] - 0s 85us/sample - loss: 0.4714 - acc: 0.8025
Epoch 7/20
891/891 [==============================] - 0s 57us/sample - loss: 0.4607 - acc: 0.7969
Epoch 8/20
891/891 [==============================] - 0s 71us/sample - loss: 0.4579 - acc: 0.7901
Epoch 9/20
891/891 [==============================] - 0s 71us/sample - loss: 0.4452 - acc: 0.8047
Epoch 10/20
891/891 [==============================] - 0s 69us/sample - loss: 0.4432 - acc: 0.81

In [109]:
submission_list = []
for i in preds:
    if i[0] > i[1]:
        submission_list.append(0)
    else:
        submission_list.append(1)
        
titanic_submission['Survived'] = submission_list
titanic_submission.to_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/titanic/submission.csv', index=False)

I submitted this to Kaggle, and got a score of .75119 accuracy. This beats my previous score of .72 from my frist Kaggle project over a year ago which did not use neural networks but had considerably more effort spent on cleaning and feature engineering. I guess just throwing a neural network at it was better!

<h1>Handwritten Digit Recognizer</h1>

In [184]:
digit_train = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/digit-recognizer/train.csv')
digit_test = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/digit-recognizer/test.csv')
digit_submission = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/digit-recognizer/sample_submission.csv')

In [185]:
digit_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
digit_train.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [187]:
digit_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
digit_test.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB


In [189]:
digit_submission.head()

,ImageId,Label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [190]:
X_train = digit_train.iloc[:, 1:]
y_train = digit_train.label
X_train = X_train / 255.0
digit_test = digit_test / 255.0

X_train = X_train.values.reshape(-1, 28,28, 1)
X_test = digit_test.values.reshape(-1,28,28,1)

y_train = to_categorical(y_train, num_classes=10)

In [191]:
y_train

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [193]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32, kernel_size = (3,3), padding="same", activation='relu', input_shape = (28, 28, 1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='RMSprop',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10)
preds = model.predict(X_test)
preds = np.argmax(preds, axis=1)

Train on 42000 samples
Epoch 1/10
42000/42000 [==============================] - 53s 1ms/sample - loss: 0.1850 - acc: 0.9446
Epoch 2/10
42000/42000 [==============================] - 73s 2ms/sample - loss: 0.0668 - acc: 0.9798
Epoch 3/10
42000/42000 [==============================] - 61s 1ms/sample - loss: 0.0447 - acc: 0.9867
Epoch 4/10
42000/42000 [==============================] - 76s 2ms/sample - loss: 0.0343 - acc: 0.9897
Epoch 5/10
42000/42000 [==============================] - 77s 2ms/sample - loss: 0.0265 - acc: 0.9926
Epoch 6/10
42000/42000 [==============================] - 68s 2ms/sample - loss: 0.0201 - acc: 0.9937
Epoch 7/10
42000/42000 [==============================] - 69s 2ms/sample - loss: 0.0161 - acc: 0.9957
Epoch 8/10
42000/42000 [==============================] - 61s 1ms/sample - loss: 0.0128 - acc: 0.9966
Epoch 9/10
42000/42000 [==============================] - 57s 1ms/sample - loss: 0.0100 - acc: 0.9971
Epoch 10/10
42000/42000 [==============================] - 

In [202]:
digit_submission['Label'] = preds
digit_submission.to_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/digit-recognizer/submission.csv', index=False)

Submission score to Kaggle was 0.98185, compared to my previous submission without neural networks from school of .96

I am happy with this score considering the minimal architechture and tuning I did!

<h1>Dota 2 Last Pick Predictor</h1>

Dota 2 is a competitive online game that begins with a 'draft' in which 2 teams alternate choosing and banning heroes. This dataset was scraped in 2015 (notably dated for a game like dota 2 but this is a proof-of-concept) and I will attempt to use RNNs to predict the end of a draft given the beginning.

In [772]:
data = pd.read_csv('/Users/jdobrow/Desktop/DataScience/TensorFlow Learning/dota 2 draft/capmodedata.csv')
heroes = pd.read_json('https://raw.githubusercontent.com/larryfenn/dota2drafts/master/popularity/herolookup.json')

In [773]:
name, id_num = [], []
for i in heroes.iloc[1][0]:
    name.append(i['localized_name'])
    id_num.append(i['id'])
heroes = pd.DataFrame()
heroes['ID'] = id_num
heroes['Name'] = name
heroes.loc[-1] = [108, 'Underlord']
heroes.loc[-2] = [113, 'Arc Warden']
heroes = heroes.sort_values(by=['ID'])

cols_to_drop = []
for i in range(1, 21):
    cols_to_drop.append('{}team_id'.format(i))
    cols_to_drop.append('{}is_pick'.format(i))
data.drop(['match_id', 'duration', 'radiant_win'] + cols_to_drop, 1, inplace=True)

data = data.astype('int64')
heroes['ID'] = heroes['ID'] - 1
data = data - 1

In [774]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('20hero_id', 1), data['20hero_id'], test_size=.1)
X_train, X_test = X_train.values, X_test.values
y_train, y_test = y_train.values, y_test.values
y_train = to_categorical(y_train, num_classes=113)
X_train = np.reshape(X_train, (len(X_train), 1, len(X_train[0])))
X_test = np.reshape(X_test, (len(X_test), 1, len(X_test[0])))

In [775]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(512, dropout=0.2, recurrent_dropout=0.2, input_shape = (X_train.shape[1], X_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])
model.fit(X_train, y_train, epochs=15)
preds = model.predict(X_test)

Train on 414945 samples
Epoch 1/15
414945/414945 [==============================] - 464s 1ms/sample - loss: 4.4547 - acc: 0.0287
Epoch 2/15
414945/414945 [==============================] - 399s 962us/sample - loss: 4.4447 - acc: 0.0297
Epoch 3/15
414945/414945 [==============================] - 437s 1ms/sample - loss: 4.4422 - acc: 0.0301
Epoch 4/15
414945/414945 [==============================] - 417s 1ms/sample - loss: 4.4417 - acc: 0.0300
Epoch 5/15
414945/414945 [==============================] - 378s 911us/sample - loss: 4.4420 - acc: 0.0300
Epoch 6/15
414945/414945 [==============================] - 377s 908us/sample - loss: 4.4418 - acc: 0.0299
Epoch 7/15
414945/414945 [==============================] - 371s 894us/sample - loss: 4.4416 - acc: 0.0298
Epoch 8/15
414945/414945 [==============================] - 345s 831us/sample - loss: 4.4409 - acc: 0.0300
Epoch 9/15
414945/414945 [==============================] - 347s 836us/sample - loss: 4.4407 - acc: 0.0303
Epoch 10/15
414945/

Evaluate by 'in top 10'

In [776]:
def in_top_10(match_num):
    top_10 = []
    idx_sort = np.argsort(preds[match_num])
    for i in range(len(idx_sort)):
        top_hero = idx_sort[-1 - i]
        if top_hero == y_test[match_num]:
            return True
        elif top_hero not in X_test[match_num][0]:
            top_10.append(top_hero)
            if len(top_10) == 10:
                return False

In [777]:
preds = model.predict(X_test)
correct = 0
for match_num in range(len(X_test)):
    if in_top_10(match_num):
        correct += 1
accuracy = correct/len(X_test)
accuracy

0.30719010953258863

In [799]:
x = [0] * 113
for i in list(data['20hero_id']):
    x[i] += 1

Better baseline: Predict the top 10 most picked heroes for 20th pick.

In [796]:
most_popular = np.argsort(x)[-10:]

In [797]:
correct = 0
for i in y_test:
    if i in most_popular:
        correct += 1
accuracy = correct/len(y_test)

In [798]:
accuracy

0.2358529443661208